In [ ]:
# !pip install transformers datasets evaluate accelerate

In [ ]:
import wandb
wandb.disabled = True

In [ ]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

columns = ['label', 'text']

dataset = load_dataset('csv', data_files='/content/ecommerceDataset.csv',
                        column_names=columns)
df = dataset['train'].to_pandas()

df = df.dropna().drop_duplicates()

label2id = {"Household": 0, "Books": 1, "Clothing & Accessories": 2, "Electronics": 3}

df["label"] = [label2id[label] for label in df["label"]]

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset=train_dataset.remove_columns("__index_level_0__")
test_dataset=test_dataset.remove_columns("__index_level_0__")

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

import string
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already available
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))

def remove_punctuation_stopwords(examples):
    def clean_text(text):
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove stopwords
        text = " ".join([word for word in text.split() if word.lower() not in stop_words])
        return text.lower()

    examples['text'] = [clean_text(text) for text in examples['text']]

    return examples


train_dataset = train_dataset.map(remove_punctuation_stopwords, batched=True)
test_dataset = test_dataset.map(remove_punctuation_stopwords, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Train size: 22241, Test size: 5561


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Map:   0%|          | 0/22241 [00:00<?, ? examples/s]

Map:   0%|          | 0/5561 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
import numpy as np
from transformers import DataCollatorWithPadding
import evaluate

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
id2label = {0: "Household", 1: "Books", 2: "Clothing & Accessories", 3: "Electronics"}
label2id = {"Household": 0, "Books": 1, "Clothing & Accessories": 2, "Electronics": 3}

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# import pandas
# pandas.DataFrame(tokenized_train_dataset['label']).value_counts()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/22241 [00:00<?, ? examples/s]

Map:   0%|          | 0/5561 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    report_to="none",
    output_dir="./model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=10,
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,0.288600,0.258583,0.932926
200,0.290400,0.226122,0.941018
300,0.261900,0.228609,0.937062
400,0.162200,0.190682,0.949649
500,0.180900,0.184390,0.950369
600,0.231200,0.177406,0.952706


Step,Training Loss,Validation Loss,Accuracy
100,0.288600,0.258583,0.932926
200,0.290400,0.226122,0.941018
300,0.261900,0.228609,0.937062
400,0.162200,0.190682,0.949649
500,0.180900,0.184390,0.950369
600,0.231200,0.177406,0.952706


TrainOutput(global_step=696, training_loss=0.27597855553887357, metrics={'train_runtime': 1567.2772, 'train_samples_per_second': 14.191, 'train_steps_per_second': 0.444, 'total_flos': 2946312496361472.0, 'train_loss': 0.27597855553887357, 'epoch': 1.0})

In [11]:
from transformers import AutoTokenizer

text = "micromax phone"

tokenizer = AutoTokenizer.from_pretrained("/content/model/checkpoint-696")
inputs = tokenizer(text, return_tensors="pt")

In [14]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/content/model/checkpoint-696")
with torch.no_grad():
    logits = model(**inputs).logits

In [15]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'Electronics'

In [3]:
# testing the local endpoint

import requests
text = "micromax phone"
url = "http://localhost:8000/predict"
data = {"text": text}
response = requests.post(url, json=data)
print(response.json())

{'text': 'micromax phone', 'prediction': 'Electronics'}
